In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fashionmnist


In [3]:
import os

for file in os.listdir(path):
    print(file)


t10k-labels-idx1-ubyte
t10k-images-idx3-ubyte
fashion-mnist_test.csv
fashion-mnist_train.csv
train-labels-idx1-ubyte
train-images-idx3-ubyte


In [4]:
import pandas as pd

train_path = os.path.join(path, "fashion-mnist_train.csv")
df = pd.read_csv(train_path)

# Show the table
df.head()


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df.drop(columns="label")
y = df["label"]

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
X_train = X_train/255.0
X_test  = X_test/255.0

In [8]:
#Dataset

class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features = torch.tensor(features.values,dtype=torch.float32)
    self.labels  = torch.tensor(labels.values,dtype=torch.long)

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self,index):
    return self.features[index],self.labels[index]

In [9]:
#Create train_dataset and test_dataset

train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)

In [10]:
#Data Set Loader

train_loader = DataLoader(train_dataset,shuffle=True,batch_size=32)
test_loader = DataLoader(test_dataset,shuffle=False,batch_size=32)


In [11]:
#NN Module


class MyCustomNN(nn.Module):

  def __init__(self,input):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(input,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10),
    )


  def forward_pass(self,features):
    return self.model(features)

In [12]:
## learning parameters

epochs = 100
learning_rate = 0.1


In [13]:
model = MyCustomNN(X_train.shape[1])



loss_function = nn.CrossEntropyLoss()


optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,)




In [14]:
#loop


for epoch in range(epochs):
  total_epoch_Loss = 0
  for batch_features,batch_labels in train_loader:
    y_pred = model.forward_pass(batch_features)
    loss = loss_function(y_pred,batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_epoch_Loss = total_epoch_Loss+loss.item()

  avg_loss = total_epoch_Loss/len(train_loader)
  print(f"Epoch:{epoch+1}, loss:{avg_loss}")

Epoch:1, loss:0.6299713642100493
Epoch:2, loss:0.42717060187458994
Epoch:3, loss:0.37991291885077955
Epoch:4, loss:0.3553198474595944
Epoch:5, loss:0.3335484495609999
Epoch:6, loss:0.3165401453897357
Epoch:7, loss:0.3037376653452714
Epoch:8, loss:0.2909486527418097
Epoch:9, loss:0.28059696696698666
Epoch:10, loss:0.27176739419500034
Epoch:11, loss:0.26206646575157844
Epoch:12, loss:0.25559726483623185
Epoch:13, loss:0.2486119606209298
Epoch:14, loss:0.24182933794210354
Epoch:15, loss:0.2346522024696072
Epoch:16, loss:0.2312146554502348
Epoch:17, loss:0.2224155081336697
Epoch:18, loss:0.21895106274386247
Epoch:19, loss:0.21412962501496077
Epoch:20, loss:0.20823255546204747
Epoch:21, loss:0.2037510725830992
Epoch:22, loss:0.1968384024935464
Epoch:23, loss:0.1941688342038542
Epoch:24, loss:0.18909025121107698
Epoch:25, loss:0.18755946789806088
Epoch:26, loss:0.1836571111412098
Epoch:27, loss:0.17876085065243144
Epoch:28, loss:0.17361670595469575
Epoch:29, loss:0.17340454118512572
Epoch:30

In [15]:
model.eval()

MyCustomNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [17]:
#calculating accuracy

total = 0
correct = 0

with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    y_pred  = model.forward_pass(batch_features)
    _, predicted =  torch.max(y_pred,1)
    total +=  batch_labels.shape[0]
    correct = correct+ (predicted==batch_labels).sum().item()
  print(correct/total)



0.8799166666666667
